In [49]:
import pandas as pd
import numpy as np

data_proc = pd.read_csv('proc_group_data.csv')
data = pd.read_csv('groups_test_vals.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'proc_group_data.csv'

In [ ]:
data_proc

,student,proj_a,proj_b,proj_c,proj_d,proj_e
0,joao,3,4,5,6,7
1,afonso,6,2,3,4,5
2,margarida,7,8,4,5,6
3,pedro,7,8,9,5,6
4,nuno,7,8,9,10,6
5,miguel,9,10,11,7,8
6,francisco,12,11,8,9,10
7,ze,10,12,11,13,14


In [ ]:
data

,student,proj_0,proj_1,proj_2,proj_3,proj_4
0,joao,1,2,3,4,5
1,afonso,5,1,2,3,4
2,margarida,4,5,1,2,3
3,pedro,3,4,5,1,2
4,nuno,2,3,4,5,1
5,miguel,3,4,5,1,2
6,francisco,5,4,1,2,3
7,ze,1,3,2,4,5


In [ ]:
pref_array = data.values
pref_array = pref_array[:, 1:]

In [ ]:
pref_array.shape

(8, 5)

In [ ]:
selections_array = np.zeros(pref_array.shape).astype(int)

In [ ]:
selections_array

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [98]:
# minimize group points (maximize group preference) 
# subject to: each student is in exactly one group
#             each group has exactly 3 students
#             80% of students get their first or 2nd choice
#             100% of students get their first, 2nd, or 3rd choice

# objective function
# minimize sum of group points
# group points = sum of student preferences in group
# student preferences = 1 if student is in group, 0 otherwise

# constraints
# each student is in exactly one group
# each group has exactly 3 students
# 80% of students get their first or 2nd choice
# 100% of students get their first, 2nd, or 3rd choice

# variables
# binary variable for each student in each group
# binary variable for each group

# Path: model_test.ipynb
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

data = pd.read_csv('groups_test_vals.csv')

pref_array = data.values
pref_array = pref_array[:, 1:]

selections_array = np.zeros(pref_array.shape).astype(int)

model = gp.Model('grouping')

# add variables
# student_group_vars[i, j] = 1 if student i is in group j, 0 otherwise

student_group_vars = model.addVars(pref_array.shape[0], pref_array.shape[1], vtype=GRB.BINARY, name='student_group')
student_group_vars
# group_vars[j] = 1 if group j is used, 0 otherwise

group_vars = model.addVars(pref_array.shape[1], vtype=GRB.BINARY, name='group')

# add constraints
# each student is in exactly one group

model.addConstrs((student_group_vars.sum(i, '*') == 1 for i in range(pref_array.shape[0])), name='student_in_one_group')

# each group has exactly 3 students

model.addConstrs((student_group_vars.sum('*', j) == 3 for j in range(pref_array.shape[1])), name='three_students_per_group')

pref_dict = dict(enumerate(pref_array.flatten()))

# add objective function
# minimize sum of group points
# group points = sum of student preferences in group
# student preferences = 1 if student is in group, 0 otherwise

model.setObjective(student_group_vars.prod(), GRB.MINIMIZE)

# optimize model

model.optimize()

# get results

# for i in range(pref_array.shape[0]):
#     for j in range(pref_array.shape[1]):
#         selections_array[i, j] = student_group_vars[i, j].student_group

TypeError: prod() takes at least 2 positional arguments (1 given)